In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import struct
import hashlib
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import tensorflow_hub as hub
import tokenization
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.cluster import KMeans
import pickle
from sknetwork.ranking import PageRank
from nltk.stem.snowball import EnglishStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import torch
import sys

In [2]:
def tensor_to_string(x):
    return x.numpy().decode('UTF-8')

In [3]:
def get_sent_list(text,stem=None):
    sents = sent_tokenize(text)
    if stem == "None":
        return sents
    if stem == "EnglishStemmer":
        stemmer = EnglishStemmer()
    ans = []
    for sent in sents:
        words = word_tokenize(sent)
        word_stem = [stemmer.stem(w) for w in words]
        ans.append(str(" ".join(str(word_stem))))
    return ans

In [5]:
## Model features include an encode function -> takes a list of sentences. Returns a list of embeddings (all same dim)
transformers = ["all-mpnet-base-v2",
                "multi-qa-mpnet-base-dot-v1",
                "all-distilroberta-v1",
                "all-MiniLM-L12-v2",
                "multi-qa-distilbert-cos-v1",
                "all-MiniLM-L6-v2",
                "multi-qa-MiniLM-L6-cos-v1",
                "paraphrase-multilingual-mpnet-base-v2",
                "paraphrase-albert-small-v2",
                "paraphrase-multilingual-MiniLM-L12-v2",
                "paraphrase-MiniLM-L3-v2",
                "distiluse-base-multilingual-cased-v1",
                "distiluse-base-multilingual-cased-v2"]

models = dict()
device = torch.device("cpu")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for trans in transformers:
    models[trans] = SentenceTransformer(trans,cache_folder='/mnt/disks/disk-1/data/models')
    models[trans]._target_device = device

In [6]:
## V = list of embeddings. k = target size of summary
## Returns a list of sentence indices

def generate_summary(V, k):
    if k >= len(V):
        return list(range(len(V)))
    k -= 1
    centers = []
    cities = list(range(len(V)))
    centers.append(0)
    cities.remove(0)
    while k!= 0:
        city_dict = {}
        for cty in cities:
            min_dist = float("inf")
            for c in centers:
                min_dist = min(min_dist,np.linalg.norm(V[cty] - V[c]))
            city_dict[cty] = min_dist
        new_center = max(city_dict, key = lambda i: city_dict[i])
        centers.append(new_center)
        cities.remove(new_center)
        k -= 1
    return centers

In [7]:
## Pagerank version

def sim(a, b):
    return np.dot(a, b) / np.sqrt(np.dot(a, a) * np.dot(b, b))

def generate_summary(V,k):
    if k >= len(V):
        return list(range(len(V)))
    n = V.shape[0]
    adj = np.zeros((n, n))
    for i in range(n):
        adj[i][i] = sim(V[i],V[i])
        for j in range(i+1,n):
            s = sim(V[i], V[j])
            adj[i][j] = s
            adj[j][i] = s

    pr = PageRank()
    scores = pr.fit_transform(adj)
    ind = np.argpartition(scores, -k)[-k:]
    return np.sort(ind)

In [8]:
def uml_summary(x,index,kind="cnn_dailymail",model="all-MiniLM-L6-v2"):
    if kind == "cnn_dailymail":
        key1 = 'article'
        key2 = 'highlights'
    elif kind == "scientific_papers/arxiv" or kind == "scientific_papers/pubmed":
        key1 = 'article'
        key2 = 'abstract'
    text = tensor_to_string(x[key1])
    text = get_sent_list(text,stemmer)
    summary = tensor_to_string(x[key2])
    summary = get_sent_list(summary,stemmer)
    
    filename = str(index) + "_" + model + ".pickle"
    folderpath = os.path.join("/mnt/disks/disk-1/data/pickle",kind)
    filepath = os.path.join("/mnt/disks/disk-1/data/pickle",kind,filename)
    if os.path.exists(filepath):
        with open(filepath, 'rb') as handle:
            text_emb = pickle.load(handle)
    else:
        text_emb = models[model].encode(text)
        with open(filepath, 'wb') as handle:
            pickle.dump(text_emb, handle, protocol=pickle.HIGHEST_PROTOCOL)

    gen_sum = [text[x] for x in generate_summary(text_emb,len(summary))]
    scores = scorer.score(" ".join(summary)," ".join(gen_sum))
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

In [9]:
%%time

datasets = ["cnn_dailymail","scientific_papers/arxiv","scientific_papers/pubmed"]
datasets = ["cnn_dailymail"]
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)

for ds in datasets:
    for trans in transformers:
        train, val, test = tfds.load(name=ds, 
                              split=["train", "validation", "test"], 
                              data_dir="/mnt/disks/disk-1/data")
        
#         model = SentenceTransformer(trans,cache_folder='/mnt/disks/disk-1/data/models')
#         model._target_device = device
        r1 = []
        r2 = []
        rl = []
        index = 0
        for x in list(test):
            r1_val,r2_val,rl_val = uml_summary(x,index,kind=ds,model=trans)
            index += 1
            r1.append(r1_val)
            r2.append(r2_val)
            rl.append(rl_val)
            if index > 100:
                print(index)
        print(ds,trans)
        print(index)
        print("Rouge 1 : ",np.round(np.mean(np.asarray(r1))*100,2))
        print("Rouge 2 : ",np.round(np.mean(np.asarray(r2))*100,2))
        print("Rouge L : ",np.round(np.mean(np.asarray(rl))*100,2))
        print("___")

INFO:absl:Load dataset info from /mnt/disks/disk-1/data/cnn_dailymail/3.1.0
INFO:absl:Reusing dataset cnn_dailymail (/mnt/disks/disk-1/data/cnn_dailymail/3.1.0)
INFO:absl:Constructing tf.data.Dataset cnn_dailymail for split ['train', 'validation', 'test'], from /mnt/disks/disk-1/data/cnn_dailymail/3.1.0
2021-11-28 22:45:22.056961: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-28 22:45:22.534529: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38464 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0
2021-11-28 22:45:22.969998: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185]

101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


KeyboardInterrupt: 

In [20]:
for file in ["kmeans.py","pagerank.py"]:
    for model in transformers:
        for ds in datasets:
            for stem in ["None","EnglishStemmer"]:
                print("python",file,'"' + model + '"','"' + ds + '"','"' + stem + '"')

python kmeans.py "all-mpnet-base-v2" "cnn_dailymail" "None"
python kmeans.py "all-mpnet-base-v2" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "multi-qa-mpnet-base-dot-v1" "cnn_dailymail" "None"
python kmeans.py "multi-qa-mpnet-base-dot-v1" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "all-distilroberta-v1" "cnn_dailymail" "None"
python kmeans.py "all-distilroberta-v1" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "all-MiniLM-L12-v2" "cnn_dailymail" "None"
python kmeans.py "all-MiniLM-L12-v2" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "multi-qa-distilbert-cos-v1" "cnn_dailymail" "None"
python kmeans.py "multi-qa-distilbert-cos-v1" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "all-MiniLM-L6-v2" "cnn_dailymail" "None"
python kmeans.py "all-MiniLM-L6-v2" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "multi-qa-MiniLM-L6-cos-v1" "cnn_dailymail" "None"
python kmeans.py "multi-qa-MiniLM-L6-cos-v1" "cnn_dailymail" "EnglishStemmer"
python kmeans.py "paraphrase-multi